# Devoir maison :  analyse de schémas JSON publiques - Décembre 2019 (version simplifiée)

## Description 

Le but de ce travail est d'analyser des schémas JSON exprimés en JSON Schema afin d'en extraire des statistiques sur l'usage des différentes features, en particulier les objets, les arrays et les contraintes.
La spécification officelle se trouve ici

* https://json-schema.org

Un tutoriel 

* https://json-schema.org/understanding-json-schema/index.html

Les schémas se trouvent dans

* https://github.com/SchemaStore/schemastore

sous 

`schemastore/src/schemas/json/`

Plus précisément, il est demandé d'extraire, pour un schéma donné, les informations suivantes :
* le nombre d'object définis dans le schéma. Pour ce faire, il suffit de compter le nombre de fois où l'attribut `type` prend la valeur `object`. Attention, cet attribut peut apparaitre à n'importe quelle profondeur (cf. exemple qui suit)
* le nombre de fields de tous les objects, il s'agit de collecter le nombre de fields apparaissant à l'interieur du field `properties`. Idem que pour `type`, `properties` peut apparaitre à n'importe quelle profondeur (cf. exemple qui suit)
* le nombre de `required fields` de tous les objets, il s'agit de collecter, pour chaque field `required`, la taille du tableau qui lui est associé.


Le format de sortie exigé est un objet JSON avec quatre attributs:
* schema : de type String, contient le nom du fichier du schéma analysé
* nbObj : de type Int, contient le nombre d'objects
* fieldDist : de type Array[Int], contient les nombres de fields des objets
* requiredDist : de type Array[Int], contient les nombres de fields required


A titre d'exemple, considérons le schéma `apple-app-site-association.json` pour lequel votre programme doit retourner 

`{"schema": "apple-app-site-association.json", "nbObj": 3, "fieldDist": [1,2,2], "requiredDist":[2,1]}`

En effet, il y a bien 3 fois `type="object"` en examinant ces paths
* type
* properties.applinks.type
* properties.applinks.properties.details.items.type

Il y a également deux paths pour accéder au field `required` qui sont :
* required
* properties.applinks.required

Pour faciliter la réalisation du projet, il est conseiller commencer par extraire touts les chemins à partir du schéma du dataframe obtenu en chargeant votre fichier json. Ces chemins vous servirons à formuler vos requête dataframe pour réaliser les test.

Une architecture du programme est suggérée dans ce notebook. Vous pouvez la suivre scrupuleusement ou vous en inspirer pour écrire votre programe. 

### Deadline

A rendre pour le 20-12-2019 18H00 par envoi de mail à baazizi@ia.lip6.fr. Objet : [BDLE-DS-Dec19]

Envoyer code + commentaire ou lien publique github contenant code + résultat 

### Implantation

Conseillé d'utiliser dataframe.

Test sur 2 schemas du repository au choix : 1 petit (<100 KB) et un grand (>100 KB)


## Code

Rappel des liens de l'API Dataset 
* https://spark.apache.org/docs/latest/sql-programming-guide.html
* https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Dataset
* https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$

In [2]:
val path = "C:/Users/yousef/Desktop/Courses/json/"
//"/tmp/BDLE/dataset/schemastore/"

path: String = C:/Users/yousef/Desktop/Courses/json/


In [28]:
//val file = "vega.json"
//val file = "appsettings.json"
val file="circleciconfig.json"
//val file = "apple-app-site-association.json"
val data = spark.read.format("json").option("multiLine",true).load(path+file)
//data.printSchema

file: String = circleciconfig.json
data: org.apache.spark.sql.DataFrame = [$schema: string, definitions: struct<builtinSteps: struct<configuration: struct<add_ssh_keys: struct<additionalProperties: boolean, allOf: array<struct<$ref:string>> ... 2 more fields>, attach_workspace: struct<additionalProperties: boolean, allOf: array<struct<$ref:string>> ... 3 more fields> ... 9 more fields>, documentation: struct<add_ssh_keys: struct<description: string>, attach_workspace: struct<description: string> ... 9 more fields>>, commands: struct<additionalProperties: struct<description: string, properties: struct<description: struct<description: string, type: string>, parameters: struct<description: string, patternProperties: struct<^[a-z][a-z0-9_-]+$: struct<oneOf: array<struct<description:string,...

In [29]:
data.schema.fields

res10: Array[org.apache.spark.sql.types.StructField] = Array(StructField($schema,StringType,true), StructField(definitions,StructType(StructField(builtinSteps,StructType(StructField(configuration,StructType(StructField(add_ssh_keys,StructType(StructField(additionalProperties,BooleanType,true), StructField(allOf,ArrayType(StructType(StructField($ref,StringType,true)),true),true), StructField(properties,StructType(StructField(fingerprints,StructType(StructField(description,StringType,true), StructField(items,StructType(StructField(type,StringType,true)),true), StructField(type,StringType,true)),true), StructField(name,StructType(StructField(description,StringType,true), StructField(type,StringType,true)),true)),true), StructField(type,StringType,true)),true), StructField(attach_workspace,...

In [30]:
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._


import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._


### Extract all paths from the schema

In [5]:
val _sep = "." 
def fieldNames(path: Seq[String], schema: DataType): Seq[String] = {
    schema match {
        case s: StructType => s.fields.flatMap( f => fieldNames(path++Seq(f.name), f.dataType) )
        case a: ArrayType => fieldNames(path, a.elementType)
        case _ =>  Seq(path.mkString(_sep))
      }
}

_sep: String = .
fieldNames: (path: Seq[String], schema: org.apache.spark.sql.types.DataType)Seq[String]


In [31]:
/*fieldNames est invoquée initialement avec un chemin vide*/
val fnames = fieldNames(Nil, data.schema)
fnames.foreach(println)


$schema
definitions.builtinSteps.configuration.add_ssh_keys.additionalProperties
definitions.builtinSteps.configuration.add_ssh_keys.allOf.$ref
definitions.builtinSteps.configuration.add_ssh_keys.properties.fingerprints.description
definitions.builtinSteps.configuration.add_ssh_keys.properties.fingerprints.items.type
definitions.builtinSteps.configuration.add_ssh_keys.properties.fingerprints.type
definitions.builtinSteps.configuration.add_ssh_keys.properties.name.description
definitions.builtinSteps.configuration.add_ssh_keys.properties.name.type
definitions.builtinSteps.configuration.add_ssh_keys.type
definitions.builtinSteps.configuration.attach_workspace.additionalProperties
definitions.builtinSteps.configuration.attach_workspace.allOf.$ref
definitions.builtinSteps.configuration.attach_workspace.properties.at.description
definitions.builtinSteps.configuration.attach_workspace.properties.at.type
definitions.builtinSteps.configuration.attach_workspace.properties.name.description
defin

definitions.dockerExecutor.items.properties.command.description
definitions.dockerExecutor.items.properties.command.oneOf.items.type
definitions.dockerExecutor.items.properties.command.oneOf.type
definitions.dockerExecutor.items.properties.entrypoint.description
definitions.dockerExecutor.items.properties.entrypoint.oneOf.items.type
definitions.dockerExecutor.items.properties.entrypoint.oneOf.type
definitions.dockerExecutor.items.properties.environment.additionalProperties.type
definitions.dockerExecutor.items.properties.environment.description
definitions.dockerExecutor.items.properties.environment.type
definitions.dockerExecutor.items.properties.image.description
definitions.dockerExecutor.items.properties.image.type
definitions.dockerExecutor.items.properties.name.description
definitions.dockerExecutor.items.properties.name.type
definitions.dockerExecutor.items.properties.user.description
definitions.dockerExecutor.items.properties.user.type
definitions.dockerExecutor.items.required

fnames: Seq[String] = ArraySeq($schema, definitions.builtinSteps.configuration.add_ssh_keys.additionalProperties, definitions.builtinSteps.configuration.add_ssh_keys.allOf.$ref, definitions.builtinSteps.configuration.add_ssh_keys.properties.fingerprints.description, definitions.builtinSteps.configuration.add_ssh_keys.properties.fingerprints.items.type, definitions.builtinSteps.configuration.add_ssh_keys.properties.fingerprints.type, definitions.builtinSteps.configuration.add_ssh_keys.properties.name.description, definitions.builtinSteps.configuration.add_ssh_keys.properties.name.type, definitions.builtinSteps.configuration.add_ssh_keys.type, definitions.builtinSteps.configuration.attach_workspace.additionalProperties, definitions.builtinSteps.configuration.attach_workspace.allOf.$ref, d...

### Fonctions auxiliaires

In [11]:
def suffix(s:String)= {
    // à compléter 
    val arr=s.split('.')
    arr(arr.size-1)
}

suffix: (s: String)String


In [12]:
suffix("properties.applinks.properties.details.type")


res5: String = type


In [13]:
/*lastButTwo extrait le n-2 eme attribut d'un chemin en notation pointée*/
def lastButTwo(s:String) = {
   // à compléter 
    val arr=s.split('.')
    if( arr.size > 2 ){
        arr(arr.size-3)
      } else {
        s
      }
    
}

lastButTwo: (s: String)String


In [14]:
lastButTwo("properties.applinks.properties.details.type")

res6: String = properties


In [15]:
/*removeTwoLasts retire les 2 derniers attributs d'un chemin en notation pointée*/
def removeTwoLasts(s:String)= {
   // à compléter 
    val arr=s.split('.')
    if( arr.size > 2 ){
         arr.slice(0, arr.size-2).reduce((a,b)=>a+"."+b)
      } else {
         s
      }
    

}

removeTwoLasts: (s: String)String


In [141]:
removeTwoLasts("properties.applinks.properties.details.type")



res109: String = properties.applinks.properties


In [12]:
removeTwoLasts("properties")


res5: String = properties


### Extraire le nombre d'objets

In [33]:
val newDf = data.withColumn("station_id_str", data.col("definitions.builtinSteps.configuration.restore_cache.oneOf.properties.key.type").cast(StringType))

newDf: org.apache.spark.sql.DataFrame = [$schema: string, definitions: struct<builtinSteps: struct<configuration: struct<add_ssh_keys: struct<additionalProperties: boolean, allOf: array<struct<$ref:string>> ... 2 more fields>, attach_workspace: struct<additionalProperties: boolean, allOf: array<struct<$ref:string>> ... 3 more fields> ... 9 more fields>, documentation: struct<add_ssh_keys: struct<description: string>, attach_workspace: struct<description: string> ... 9 more fields>>, commands: struct<additionalProperties: struct<description: string, properties: struct<description: struct<description: string, type: string>, parameters: struct<description: string, patternProperties: struct<^[a-z][a-z0-9_-]+$: struct<oneOf: array<struct<description:string,properties:struct<default:struct<it...

In [49]:
newDf.select("station_id_str")

res27: org.apache.spark.sql.DataFrame = [station_id_str: string]


In [53]:
val x="station_id_str"
newDf.filter(x+"=='object'").count

x: String = station_id_str
res29: Long = 0


In [32]:
val typePaths = fnames.filter(suffix(_)=="type")
val djk=typePaths.filter(x=>data.filter(x+"=='object'").count==1)
// à compléter
val nbObj=djk.size

org.apache.spark.sql.AnalysisException:  cannot resolve '(`definitions`.`builtinSteps`.`configuration`.`restore_cache`.`oneOf`.`properties`.`key`.`type` = 'object')' due to data type mismatch: differing types in '(`definitions`.`builtinSteps`.`configuration`.`restore_cache`.`oneOf`.`properties`.`key`.`type` = 'object')' (array<string> and string).; line 1 pos 0;

### Extraire les required fields pour les objets

In [25]:
val reqPaths = fnames.filter(suffix(_)=="required")
val countReq =reqPaths.map(x=>data.select(x).as[Seq[String]].rdd.first.size).toArray
val requiredDist=countReq
// à compléter 

reqPaths: Seq[String] = ArraySeq()
countReq: Array[Int] = Array()
requiredDist: Array[Int] = Array()


### Extraire le nombre de fields pour les objets

In [26]:
val propPaths = fnames.filter(lastButTwo(_)=="properties").map(x=>removeTwoLasts(x)).distinct
//propPaths.foreach(println)
val fieldDist = propPaths.map(x=>data.select(col(x+".*")).columns.size) 

propPaths: Seq[String] = ArraySeq(definitions.cdn.properties, definitions.pwa.properties, definitions.webOptimizer.properties)
fieldDist: Seq[Int] = ArraySeq(2, 6, 2)


In [27]:

val fieldDist=propPaths.map(
    x=>fnames.filter(
        y=>y.startsWith(x+".")).map(z=>z.split('.')(x.split('.').size)).distinct.size
                                   )

fieldDist: Seq[Int] = ArraySeq(2, 6, 2)
